In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from photutils.aperture import CircularAperture, aperture_photometry
from reproject import reproject_interp
from pathlib import Path
from IPython.display import display

In [4]:

# --- CSV with RA/Dec ---
# Columns: ID, RA (h:m:s), DEC (d:m:s)
coords_csv = "../../Downloads/ra_dec.csv"

# --- Fixed aperture ---
r_ap_pix = 10.0             # pixels (10 px ~ 2.62 arcsec for 0.262"/px)
pixel_scale_arcsec = 0.262  # arcsec/pixel (for reference only)

# --- Paths to FITS (science in nmgy; weight as inverse-variance 1/nmgy^2) ---
path_image_g  = "../../../belen/masa/NGC1313_image_g.fits"
path_image_i  = "../../../belen/masa/NGC1313_image_i.fits"
path_weight_g = "../../../belen/decals_data/ngc1313/ngc1313_weight_g.fits"
path_weight_i = "../../../belen/decals_data/ngc1313/ngc1313_weight_i.fits"

# --- Output ---
out_csv = "phot_legacy_ap10px_ra_dec.csv"

print(f"Aperture radius: {r_ap_pix} px  (~{r_ap_pix*pixel_scale_arcsec:.2f} arcsec)")

Aperture radius: 10.0 px  (~2.62 arcsec)


In [5]:
for p in [path_image_g, path_image_i, path_weight_g, path_weight_i, coords_csv]:
    P = Path(p)
    try:
        print(p, "->", P.resolve())
    except Exception as e:
        print(p, "->", e)
    print("  exists:", P.exists())

../../../belen/masa/NGC1313_image_g.fits -> /Users/belen/masa/NGC1313_image_g.fits
  exists: True
../../../belen/masa/NGC1313_image_i.fits -> /Users/belen/masa/NGC1313_image_i.fits
  exists: True
../../../belen/decals_data/ngc1313/ngc1313_weight_g.fits -> /Users/belen/decals_data/ngc1313/ngc1313_weight_g.fits
  exists: True
../../../belen/decals_data/ngc1313/ngc1313_weight_i.fits -> /Users/belen/decals_data/ngc1313/ngc1313_weight_i.fits
  exists: True
../../Downloads/ra_dec.csv -> /Users/belen/Downloads/ra_dec.csv
  exists: True


In [6]:

def list_2d_hdus(path):
    from astropy.io import fits
    with fits.open(path, memmap=True) as hdul:
        out = []
        for i, h in enumerate(hdul):
            shp = None if h.data is None else getattr(h.data, 'shape', None)
            if isinstance(shp, tuple) and len(shp) == 2:
                out.append((i, shp, h.name))
        return out

def load_first_2d(path):
    with fits.open(path, memmap=True) as hdul:
        for i, h in enumerate(hdul):
            if h.data is not None and isinstance(h.data, np.ndarray) and h.data.ndim == 2:
                return np.asarray(h.data, float), h.header, i
    raise ValueError(f"No 2D image HDU found in {path}")

def best_match_hdu(weight_path, target_shape):
    cands = list_2d_hdus(weight_path)
    exact = [i for (i, shp, name) in cands if shp == target_shape]
    if exact:
        return exact[0]
    # fallback: largest area
    areas = [(i, shp[0]*shp[1]) for (i, shp, name) in cands]
    if not areas:
        raise ValueError("No 2D HDUs in weight file")
    return max(areas, key=lambda t: t[1])[0]

def load_image_and_weight(image_path, weight_path):
    img, hdr_img, img_hdu = load_first_2d(image_path)
    # choose weight HDU: same shape preferred; otherwise largest 2D
    with fits.open(weight_path, memmap=True) as hw:
        w_match = best_match_hdu(weight_path, img.shape)
        ivar = np.asarray(hw[w_match].data, float)
        hdr_wgt = hw[w_match].header
    meta = {
        "img_hdu": img_hdu, "wgt_hdu": w_match,
        "img_shape": img.shape, "wgt_shape": ivar.shape
    }
    return img, hdr_img, ivar, hdr_wgt, meta

def ivar_to_sigma_and_mask(ivar):
    """
    Convert inverse-variance (1/nmgy^2) to per-pixel 1-sigma error (nmgy).
    Mask pixels where ivar <= 0 or not finite.
    """
    inv_ok = np.isfinite(ivar) & (ivar > 0)
    sigma = np.full_like(ivar, np.nan, dtype=float)
    sigma[inv_ok] = 1.0 / np.sqrt(ivar[inv_ok])
    mask = ~inv_ok  # True means masked
    return sigma, mask

def reproject_sigma_to_image_grid(sigma, hdr_sigma, hdr_image, min_footprint=0.5):
    """
    Reproject sigma map to the science image grid using WCS-based interpolation.
    Also return a mask where footprint is too low or sigma is invalid.
    """
    re_sigma, footprint = reproject_interp((sigma, hdr_sigma), hdr_image)
    mask = ~np.isfinite(re_sigma) | (footprint < min_footprint)
    return re_sigma, mask

def frac_unmasked_in_apertures(mask_bool, apertures):
    """
    Fraction of aperture area that is unmasked (1.0 means fully unmasked).
    """
    fracs = []
    for m in apertures.to_mask(method='exact'):
        cut_mask = m.cutout(mask_bool.astype(float))
        if cut_mask is None:
            fracs.append(np.nan); continue
        frac_img = m.data
        valid = 1.0 - cut_mask
        unmasked_area = np.nansum(frac_img * valid)
        total_area = np.nansum(frac_img)
        fracs.append(unmasked_area / total_area if total_area > 0 else np.nan)
    return np.array(fracs)

def ra_dec_to_pixels(df, header):
    sc = SkyCoord(df['RA'].astype(str).values, df['DEC'].astype(str).values,
                  unit=(u.hourangle, u.deg), frame='icrs')
    wcs = WCS(header)
    x, y = wcs.world_to_pixel(sc)
    return x, y

def photometry_band(image_path, weight_path, df_coords, r_pix=10.0, zp=22.5, verbose=True):
    """
    Full pipeline for one band:
    - Read science (nmgy) and weight (ivar)
    - Convert ivar -> sigma (nmgy)
    - Reproject sigma to science grid if needed
    - RA/Dec -> (x,y) via Astropy WCS
    - Aperture photometry with error propagation
    """
    img, hdr_img, ivar, hdr_wgt, meta = load_image_and_weight(image_path, weight_path)
    if verbose:
        print(f"[HDU chosen] image idx={meta['img_hdu']} shape={meta['img_shape']} | "
              f"weight idx={meta['wgt_hdu']} shape={meta['wgt_shape']}")

    # Build sigma from ivar (nmgy units)
    sigma, mask_w = ivar_to_sigma_and_mask(ivar)

    # Reproject sigma to science grid if shapes or WCS differ
    need_reproj = (img.shape != sigma.shape)
    if not need_reproj:
        try:
            w_img = WCS(hdr_img); w_wgt = WCS(hdr_wgt)
            need_reproj = not w_img.wcs.compare(w_wgt.wcs)
        except Exception:
            need_reproj = True

    if need_reproj:
        if verbose:
            print("Reprojecting σ(weight) -> image grid ...")
        sigma, mask_w = reproject_sigma_to_image_grid(sigma, hdr_wgt, hdr_img, min_footprint=0.5)

    # Combine with NaNs in the science image
    mask = mask_w | ~np.isfinite(img)

    # RA/Dec -> pixel coordinates
    x, y = ra_dec_to_pixels(df_coords, hdr_img)
    positions = list(zip(x, y))

    # Aperture photometry (no sky annulus; Legacy is sky-subtracted)
    ap = CircularAperture(positions, r=r_pix)
    tab = aperture_photometry(img, ap, error=sigma, mask=mask, method='exact')

    F = np.array(tab['aperture_sum'], dtype=float)
    sigF = np.array(tab['aperture_sum_err'], dtype=float)

    with np.errstate(divide='ignore', invalid='ignore'):
        snr = F / sigF
        mag = zp - 2.5*np.log10(F)
        mag_err = (2.5/np.log(10.0)) * (sigF / F)  # 1.085736... * sigF/F
        mag_3sig = zp - 2.5*np.log10(3.0 * sigF)
        mag_5sig = zp - 2.5*np.log10(5.0 * sigF)

    frac_unmasked = frac_unmasked_in_apertures(mask, ap)

    out = pd.DataFrame({
        'id': df_coords['ID'].values,
        'x_pix': x, 'y_pix': y,
        'flux': F, 'flux_err': sigF, 'snr': snr,
        'mag': mag, 'mag_err': mag_err,
        'mag_3sigma': mag_3sig, 'mag_5sigma': mag_5sig,
        'frac_unmasked': frac_unmasked
    })
    return out


In [8]:

df_coords = pd.read_csv(coords_csv)
assert {'ID','RA','DEC'}.issubset(df_coords.columns), "CSV must contain columns: ID, RA, DEC"
print(f"Loaded {len(df_coords)} targets.")

g_df = photometry_band(path_image_g, path_weight_g, df_coords, r_pix=r_ap_pix, zp=22.5, verbose=True)
g_df = g_df.add_prefix('g_')
display(g_df)

i_df = photometry_band(path_image_i, path_weight_i, df_coords, r_pix=r_ap_pix, zp=22.5, verbose=True)
i_df = i_df.add_prefix('i_')
display(i_df)


Loaded 19 targets.
[HDU chosen] image idx=0 shape=(1832, 1832) | weight idx=0 shape=(4397, 4397)
Reprojecting σ(weight) -> image grid ...


,g_id,g_x_pix,g_y_pix,g_flux,g_flux_err,g_snr,g_mag,g_mag_err,g_mag_3sigma,g_mag_5sigma,g_frac_unmasked
0,1,1153.497996,797.114424,221.418490,0.082133,2695.865275,16.636965,0.000403,24.020908,23.466286,1.0
1,2,1182.369139,791.887873,115.332333,0.067427,1710.468321,17.345122,0.000635,24.235107,23.680485,1.0
2,3,960.945838,757.722750,123.604484,0.068924,1793.347384,17.269914,0.000605,24.211272,23.656650,1.0
3,4,820.472994,1185.841576,148.624187,0.068719,2162.771885,17.069776,0.000502,24.214500,23.659878,1.0
4,5,723.221647,1174.194165,172.101246,0.073301,2347.858774,16.910540,0.000462,24.144417,23.589795,1.0
5,6,414.018117,994.428027,165.160066,0.067579,2443.959913,16.955237,0.000444,24.232669,23.678048,1.0
6,7,336.477147,863.020069,36.409695,0.040956,888.986855,18.596957,0.001221,24.776393,24.221771,1.0
7,8,595.949938,1100.597441,62.252560,0.052111,1194.619122,18.014607,0.000909,24.514877,23.960256,1.0
8,9,890.349382,1180.567730,115.293048,0.062118,1856.044963,17.345492,0.000585,24.324160,23.769538,1.0
9,10,855.478637,1115.840041,97.888198,0.058302,1678.997523,17.523174,0.000647,24.392996,23.838374,1.0


[HDU chosen] image idx=0 shape=(1832, 1832) | weight idx=0 shape=(4397, 4397)
Reprojecting σ(weight) -> image grid ...


,i_id,i_x_pix,i_y_pix,i_flux,i_flux_err,i_snr,i_mag,i_mag_err,i_mag_3sigma,i_mag_5sigma,i_frac_unmasked
0,1,1153.497996,797.114424,158.688473,0.133607,1187.728876,16.998637,0.000914,23.492627,22.938005,1.0
1,2,1182.369139,791.887873,125.240685,0.164894,759.524466,17.255636,0.001429,23.264188,22.709566,1.0
2,3,960.945838,757.722750,149.961819,0.132559,1131.286454,17.060048,0.000960,23.501177,22.946555,1.0
3,4,820.472994,1185.841576,115.446611,0.137428,840.049712,17.344047,0.001292,23.462006,22.907384,1.0
4,5,723.221647,1174.194165,141.624251,0.142298,995.263263,17.122156,0.001091,23.424198,22.869576,1.0
5,6,414.018117,994.428027,128.142638,0.115555,1108.936601,17.230766,0.000979,23.650230,23.095608,1.0
6,7,336.477147,863.020069,32.568564,0.100950,322.620482,18.718003,0.003365,23.796930,23.242308,1.0
7,8,595.949938,1100.597441,61.330143,0.111287,551.100871,18.030815,0.001970,23.691090,23.136468,1.0
8,9,890.349382,1180.567730,144.426396,0.143250,1008.215413,17.100884,0.001077,23.416964,22.862342,1.0
9,10,855.478637,1115.840041,129.067445,0.140123,921.101290,17.222958,0.001179,23.440924,22.886302,1.0


In [9]:

out = pd.DataFrame({'ID': df_coords['ID'].values})
out = out.merge(g_df, left_on='ID', right_on='g_id', how='left').drop(columns=['g_id'])
out = out.merge(i_df, left_on='ID', right_on='i_id', how='left').drop(columns=['i_id'])

out.to_csv(out_csv, index=False)
print("Saved:", out_csv)
display(out)

# Quality summary
n_bad_g = np.sum(out['g_frac_unmasked'] < 0.9)
n_bad_i = np.sum(out['i_frac_unmasked'] < 0.9)
print(f"Heavy masking (frac_unmasked < 0.9) — g: {n_bad_g}, i: {n_bad_i}")


Saved: phot_legacy_ap10px_ra_dec.csv


,ID,g_x_pix,g_y_pix,g_flux,g_flux_err,g_snr,g_mag,g_mag_err,g_mag_3sigma,g_mag_5sigma,...,i_x_pix,i_y_pix,i_flux,i_flux_err,i_snr,i_mag,i_mag_err,i_mag_3sigma,i_mag_5sigma,i_frac_unmasked
0,1,1153.497996,797.114424,221.418490,0.082133,2695.865275,16.636965,0.000403,24.020908,23.466286,...,1153.497996,797.114424,158.688473,0.133607,1187.728876,16.998637,0.000914,23.492627,22.938005,1.0
1,2,1182.369139,791.887873,115.332333,0.067427,1710.468321,17.345122,0.000635,24.235107,23.680485,...,1182.369139,791.887873,125.240685,0.164894,759.524466,17.255636,0.001429,23.264188,22.709566,1.0
2,3,960.945838,757.722750,123.604484,0.068924,1793.347384,17.269914,0.000605,24.211272,23.656650,...,960.945838,757.722750,149.961819,0.132559,1131.286454,17.060048,0.000960,23.501177,22.946555,1.0
3,4,820.472994,1185.841576,148.624187,0.068719,2162.771885,17.069776,0.000502,24.214500,23.659878,...,820.472994,1185.841576,115.446611,0.137428,840.049712,17.344047,0.001292,23.462006,22.907384,1.0
4,5,723.221647,1174.194165,172.101246,0.073301,2347.858774,16.910540,0.000462,24.144417,23.589795,...,723.221647,1174.194165,141.624251,0.142298,995.263263,17.122156,0.001091,23.424198,22.869576,1.0
5,6,414.018117,994.428027,165.160066,0.067579,2443.959913,16.955237,0.000444,24.232669,23.678048,...,414.018117,994.428027,128.142638,0.115555,1108.936601,17.230766,0.000979,23.650230,23.095608,1.0
6,7,336.477147,863.020069,36.409695,0.040956,888.986855,18.596957,0.001221,24.776393,24.221771,...,336.477147,863.020069,32.568564,0.100950,322.620482,18.718003,0.003365,23.796930,23.242308,1.0
7,8,595.949938,1100.597441,62.252560,0.052111,1194.619122,18.014607,0.000909,24.514877,23.960256,...,595.949938,1100.597441,61.330143,0.111287,551.100871,18.030815,0.001970,23.691090,23.136468,1.0
8,9,890.349382,1180.567730,115.293048,0.062118,1856.044963,17.345492,0.000585,24.324160,23.769538,...,890.349382,1180.567730,144.426396,0.143250,1008.215413,17.100884,0.001077,23.416964,22.862342,1.0
9,10,855.478637,1115.840041,97.888198,0.058302,1678.997523,17.523174,0.000647,24.392996,23.838374,...,855.478637,1115.840041,129.067445,0.140123,921.101290,17.222958,0.001179,23.440924,22.886302,1.0


Heavy masking (frac_unmasked < 0.9) — g: 0, i: 0


In [10]:
# --- CSV minimal, sólo las columnas pedidas y en el orden solicitado ---
cols = ['ID', 'i_mag', 'i_mag_err', 'g_mag', 'g_mag_err']
missing = [c for c in cols if c not in out.columns]
if missing:
    raise KeyError(f"Faltan columnas en 'out': {missing}")

minimal = out[cols].copy()
minimal = minimal.rename(columns={
    'i_mag':'mag_i', 'i_mag_err':'err_i',
    'g_mag':'mag_g', 'g_mag_err':'err_g'
})

minimal.to_csv("phot_legacy_ap10px_ra_dec_MIN.csv", index=False)
print("Guardado:", "phot_legacy_ap10px_ra_dec_MIN.csv")
display(minimal)


Guardado: phot_legacy_ap10px_ra_dec_MIN.csv


,ID,mag_i,err_i,mag_g,err_g
0,1,16.998637,0.000914,16.636965,0.000403
1,2,17.255636,0.001429,17.345122,0.000635
2,3,17.060048,0.000960,17.269914,0.000605
3,4,17.344047,0.001292,17.069776,0.000502
4,5,17.122156,0.001091,16.910540,0.000462
5,6,17.230766,0.000979,16.955237,0.000444
6,7,18.718003,0.003365,18.596957,0.001221
7,8,18.030815,0.001970,18.014607,0.000909
8,9,17.100884,0.001077,17.345492,0.000585
9,10,17.222958,0.001179,17.523174,0.000647
